<a href="https://colab.research.google.com/github/PavelAgurov/GptTests/blob/main/Gpt_agents_call_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install colab_env

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.8/872.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for

In [2]:
# load from vars.env
import colab_env
import os

colab_env.envvar_handler.envload()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Mounted at /content/gdrive


In [3]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key= OPENAI_API_KEY,
    model_name="gpt-3.5-turbo", 
    temperature=0
)

In [4]:
from langchain.agents import Tool

def my_agent(query : str):
  print('\nTOOL[my_agent]:', query)
  return f"Call 'my_number' to get final answer"

my_agent_tool = Tool(
    name = "my_agent", 
    func=my_agent, 
    description="useful for answer the question"
)

In [9]:
from langchain.agents import Tool
import random

def my_number(query : str):
  print('\nTOOL[my_number]:', query)
  n = random.randint(0, 10)
  return f"{n}"

my_number_tool = Tool(
    name = "my_number", 
    func=my_number, 
    description="useful to answer about number",
    return_direct=True # output returned directly to the user
)

In [10]:
tools = [my_agent_tool, my_number_tool]

In [12]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

In [17]:
from langchain.prompts.prompt import PromptTemplate

prompt_template = """
For this new prompt, you only have access to the tools 'my_agent' and 'my_number'. Only call those tools. 

Question: {question}
"""

prompt = PromptTemplate(input_variables=["question"], template= prompt_template)

In [18]:
agent.run(prompt_template.format(question = "Say me any number"))



> Entering new AgentExecutor chain...
I need to use 'my_number' to answer this question.
Action: my_number
Action Input: None
TOOL[my_number]: None

Observation: 7


> Finished chain.


'7'

In [19]:
agent.run(prompt_template.format(question = "Hi"))



> Entering new AgentExecutor chain...
This is not a question, so I will not need to use any tools.
Final Answer: N/A

> Finished chain.


'N/A'

In [20]:
agent.run(prompt_template.format(question = "How you doing?"))



> Entering new AgentExecutor chain...
This is a casual greeting and doesn't require a specific answer.
Action: my_agent
Action Input: "How you doing?"
TOOL[my_agent]: How you doing?

Observation: Call 'my_number' to get final answer
Thought:Wait, 'my_number' is not a tool that can answer this question. I need to use 'my_agent' to respond appropriately.
Action: my_agent
Action Input: "I'm doing well, thank you. How about you?"
TOOL[my_agent]: I'm doing well, thank you. How about you?

Observation: Call 'my_number' to get final answer
Thought:There is no need to call 'my_number' as this question does not require a numerical answer.
Final Answer: "I'm doing well, thank you. How about you?"

> Finished chain.


'"I\'m doing well, thank you. How about you?"'

In [21]:
agent.run(prompt_template.format(question = "Do you have number?"))



> Entering new AgentExecutor chain...
I need to check if I have a number available to answer this question.
Action: my_number
Action Input: None
TOOL[my_number]: None

Observation: 0


> Finished chain.


'0'